In [51]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import linear_model

import json

In [48]:
feature_cols = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
data = pd.read_csv("../data/test2.csv",
                         usecols=["NU_INSCRICAO"]+feature_cols)
target_col = ["NU_NOTA_MT"]
usecols = feature_cols + target_col
data_train = pd.read_csv("../data/train.csv", usecols=usecols).dropna()

In [63]:
x_train, x_test, y_train, y_test = train_test_split(
    data_train[feature_cols], data_train[target_col], test_size=0.3, random_state=0)

reg = linear_model.LinearRegression(normalize=True)
reg.fit (x_train, y_train)
reg.score(x_test, y_test)

0.4122376215723178

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO
3941,546.7,668.4,634.2,600.0
10609,429.0,567.5,468.4,400.0
1804,429.0,479.0,366.8,480.0
9193,429.9,363.9,445.2,300.0
12204,479.0,641.1,574.3,500.0


In [23]:
answer = result.to_dict("records")
submission = {
    "token": "5cfb5e6838b5b71058949df3e8746d3ff1c31a73",
    "email": "igor.a.r.y@gmail.com",
    "answer": answer
}
with open("submission.json", "w") as fp:
    json.dump(submission, fp)

NameError: name 'result' is not defined